# Komplexe Wechselstromrechnung

In [1]:
# Bibliotheken importieren
import ipywidgets as widgets
from ipywidgets import Box, HBox, VBox, Layout, Image
from ipycanvas import MultiCanvas, Canvas, hold_canvas
from math import pi, sin, cos, atan, sqrt

# Farben definieren
colors = {
    "black": "#000000",
    "U"    : "#3333cc",
    "I"    : "#e8082c",
    "R"    : "#00d000"
}

# Variablen initiieren
alpha = 0
phi = 0
animation_time = 0

# LaTeX-Symbole in Canvas vorzeichnen
img_Re = Canvas(width=22, height=12)
img_Re.draw_image(Image.from_file("img/dia_Re.png"), 0, 0)
img_j_Im = Canvas(width=35, height=16)
img_j_Im.draw_image(Image.from_file("img/dia_j_Im.png"), 0, 0)
img_I = Canvas(width=18, height=14)
img_I.draw_image(Image.from_file("img/dia_I.png"), 0, 0)
img_I_C = Canvas(width=22, height=18)
img_I_C.draw_image(Image.from_file("img/dia_I_C.png"), 0, 0)
img_I_L = Canvas(width=22, height=18)
img_I_L.draw_image(Image.from_file("img/dia_I_L.png"), 0, 0)
img_I_R = Canvas(width=22, height=17)
img_I_R.draw_image(Image.from_file("img/dia_I_R.png"), 0, 0)
img_i_t = Canvas(width=22, height=16)
img_i_t.draw_image(Image.from_file("img/dia_i_t.png"), 0, 0)
img_omega_t = Canvas(width=23, height=11)
img_omega_t.draw_image(Image.from_file("img/dia_omega_t.png"), 0, 0)
img_U = Canvas(width=18, height=14)
img_U.draw_image(Image.from_file("img/dia_U.png"), 0, 0)
img_U_C = Canvas(width=27, height=18)
img_U_C.draw_image(Image.from_file("img/dia_U_C.png"), 0, 0)
img_U_L = Canvas(width=26, height=18)
img_U_L.draw_image(Image.from_file("img/dia_U_L.png"), 0, 0)
img_U_R = Canvas(width=26, height=18)
img_U_R.draw_image(Image.from_file("img/dia_U_R.png"), 0, 0)
img_u_t = Canvas(width=27, height=16)
img_u_t.draw_image(Image.from_file("img/dia_u_t.png"), 0, 0)

In [2]:
class KWSR_Diagramm():
    
    def __init__(self, lx, ly, layers, xlabel=None, ylabel=None):
        self.canvas = MultiCanvas(layers, width=lx, height=ly)
        self.draw_background()
        self.redraw()
        
    def draw_arrow(self, layer, xy, angle, length, color, line_width=2, tip_width=10):
        sin_a = sin((angle + 90) * pi / 180)
        cos_a = cos((angle + 90) * pi / 180)
        with hold_canvas(layer):
            layer.line_width = line_width
            layer.stroke_style = color
            layer.fill_style   = color
            layer.begin_path()
            layer.move_to(xy[0], xy[1])
            if length < 15:
                tip_length = 15 + (length - 15)
                width = tip_width / 2 * length / 15
            else:
                tip_length = 15
                width = tip_width / 2
            line_end = (xy[0] + (length - tip_length) * sin_a,
                        xy[1] + (length - tip_length) * cos_a)
            layer.line_to(line_end[0], line_end[1])
            layer.stroke()
            layer.close_path()
            # Pfeilspitze
            layer.move_to(xy[0] + length * sin_a,
                          xy[1] + length * cos_a)
            layer.line_to(line_end[0] + width * sin((angle + 180) * pi / 180),
                          line_end[1] + width * cos((angle + 180) * pi / 180)
                         )
            layer.line_to(line_end[0] - width * sin((angle + 180) * pi / 180),
                          line_end[1] - width * cos((angle + 180) * pi / 180)
                         )
            layer.fill()

In [3]:
class KWSR_Zeigerkonstruktion(KWSR_Diagramm):
    
    def __init__(self, dtype):
        self.dtype = dtype
        # Animationsparameter anhand von Diagrammtyp festlegen
        self.anim_params = {}      
        if   self.dtype == "RRL":
            self.anim_params = {
                "color1" : colors["I"],
                "color2" : colors["U"],
                "angle1" : 90,
                "angle2" : 45,
                "prefix" : -1,
                "length" : 141.4213,
                "label0" : img_I,
                "label1" : img_U_R,
                "label2" : img_U_L,
                "label3" : None,
                "label4" : img_U
            }
        elif self.dtype == "RRC":
            self.anim_params = {
                "color1" : colors["I"],
                "color2" : colors["U"],
                "angle1" : 270, 
                "angle2" : -45,
                "prefix" : 1,
                "length" : 141.4213,
                "label0" : img_I,
                "label1" : img_U_R,
                "label2" : img_U_C,
                "label3" : None,
                "label4" : img_U
            }
        elif self.dtype == "RRLC":
            self.anim_params = {
                "color1" : colors["I"],
                "color2" : colors["U"],
                "angle1" : 90,
                "angle2" : -43.0632,
                "prefix" : -1,
                "length" : 146.4548,
                "xy"     : (132, 50),
                "label0" : img_I,
                "label1" : img_U_R,
                "label2" : img_U_L,
                "label3" : img_U_C,
                "label4" : img_U
            }
        elif self.dtype == "PRL":
            self.anim_params = {
                "color1" : colors["U"],
                "color2" : colors["I"],
                "angle1" : 270,
                "angle2" : -45,
                "prefix" : 1,
                "length" : 141.4213,
                "label0" : img_U,
                "label1" : img_I_R,
                "label2" : img_I_L,
                "label3" : None,
                "label4" : img_I
            }
        elif self.dtype == "PRC":
            self.anim_params = {
                "color1" : colors["U"],
                "color2" : colors["I"],
                "angle1" : 90, 
                "angle2" : 45,
                "prefix" : -1,
                "length" : 141.4213,
                "label0" : img_U,
                "label1" : img_I_R,
                "label2" : img_I_C,
                "label3" : None,
                "label4" : img_I
            }
        elif self.dtype == "PRLC":
            self.anim_params = {
                "color1" : colors["U"],
                "color2" : colors["I"],
                "angle1" : 270,
                "angle2" : 43.0632,
                "prefix" : 1,
                "length" : 146.4548,
                "xy"     : (132, 250),
                "label0" : img_U,
                "label1" : img_I_R,
                "label2" : img_I_L,
                "label3" : img_I_C,
                "label4" : img_I
            }
        super().__init__(lx=200, ly=300, layers=6)
        
    def draw_background(self):
        # Achsen
        with hold_canvas(self.canvas[0]):
            self.draw_arrow(layer = self.canvas[0],
                            xy = (25, 150),
                            angle = 0,
                            length = 150,
                            color = colors["black"],
                            line_width = 1
                           )
            self.canvas[0].draw_image(img_Re, 165, 125)
            self.draw_arrow(layer = self.canvas[0],
                            xy = (25, 275),
                            angle = 90,
                            length = 250,
                            color = colors["black"],
                            line_width = 1
                           )
            self.canvas[0].draw_image(img_j_Im, 35, 10)
        
        # Bezugszeiger
        with hold_canvas(self.canvas[0]):
            self.draw_arrow(layer = self.canvas[0],
                            xy = (25, 157),
                            angle = 0,
                            length = 80,
                            color = self.anim_params["color1"],
                            line_width = 2
                           )
            self.canvas[0].draw_image(self.anim_params["label0"], 5, 150)

    def redraw(self):
        global animation_time

        # Erster Pfeil (R)
        if animation_time <= 250:
            with hold_canvas(self.canvas[1]):
                self.canvas[1].clear()
                self.canvas[2].clear()
                self.canvas[3].clear()
                self.canvas[4].clear()
                self.canvas[5].clear()
                self.draw_arrow(layer = self.canvas[1],
                                xy = (25, 150),
                                angle = 0,
                                length = animation_time * 100 / 250,
                                color = self.anim_params["color2"],
                                line_width = 1,
                                tip_width = 8
                                )
        elif animation_time > 250:
            with hold_canvas(self.canvas[1]):
                self.canvas[1].clear()
                self.canvas[1].draw_image(self.anim_params["label1"], 70, 125)
                self.draw_arrow(layer = self.canvas[1],
                                xy = (25, 150),
                                angle = 0,
                                length = 100,
                                color = self.anim_params["color2"],
                                line_width = 1,
                                tip_width = 8
                                )
                
        # Zweiter Pfeil (L/C)
        if 300 <= animation_time <= 550:
            with hold_canvas(self.canvas[2]):
                self.canvas[2].clear()
                self.canvas[3].clear()
                self.canvas[4].clear()
                self.draw_arrow(layer = self.canvas[2],
                                xy = (125, 150),
                                angle = self.anim_params["angle1"],
                                length = (animation_time - 300) * 100 / 250,
                                color = self.anim_params["color2"],
                                line_width = 1,
                                tip_width = 8
                                )               
        elif animation_time > 550:
            with hold_canvas(self.canvas[2]):
                self.canvas[2].clear()
                self.canvas[2].draw_image(self.anim_params["label2"], 90, 140 + self.anim_params["prefix"] * 50)
                self.draw_arrow(layer = self.canvas[2],
                                xy = (125, 150),
                                angle = self.anim_params["angle1"],
                                length = 100,
                                color = self.anim_params["color2"],
                                line_width = 1,
                                tip_width = 8
                                )

        # ggf. Dritter Pfeil (C)
        if 600 <= animation_time <= 850 and self.dtype in ["RRLC", "PRLC"]:
            with hold_canvas(self.canvas[3]):
                self.canvas[3].clear()
                self.canvas[4].clear()
                self.draw_arrow(layer = self.canvas[3],
                                xy = self.anim_params["xy"],
                                angle = -self.anim_params["angle1"],
                                length = (animation_time - 600) * 200 / 250,
                                color = self.anim_params["color2"],
                                line_width = 1,
                                tip_width = 8
                                )
        elif animation_time > 850 and self.dtype in ["RRLC", "PRLC"]:
            with hold_canvas(self.canvas[3]):
                self.canvas[3].clear()
                if self.anim_params["label3"]:
                    self.canvas[3].draw_image(self.anim_params["label3"], 140, 140 - self.anim_params["prefix"] * 50)
                self.draw_arrow(layer = self.canvas[3],
                                xy = self.anim_params["xy"],
                                angle = -self.anim_params["angle1"],
                                length = 200,
                                color = self.anim_params["color2"],
                                line_width = 1,
                                tip_width = 8
                                )

        # Resultierender Pfeil
        if 900 <= animation_time <= 1150:
            with hold_canvas(self.canvas[4]):
                self.canvas[4].clear()
                self.draw_arrow(layer = self.canvas[4],
                                xy = (25, 150),
                                angle = self.anim_params["angle2"],
                                length = (animation_time - 900) * self.anim_params["length"] / 250,
                                color = self.anim_params["color2"],
                                line_width = 3
                                )
        elif animation_time > 1150:
            with hold_canvas(self.canvas[4]):
                prefix = self.anim_params["prefix"] if len(self.dtype) == 4 else -self.anim_params["prefix"]
                self.canvas[4].clear()
                self.canvas[4].draw_image(self.anim_params["label4"], 60, 140 - prefix * 70)
                self.draw_arrow(layer = self.canvas[4],
                                xy = (25, 150),
                                angle = self.anim_params["angle2"],
                                length = self.anim_params["length"],
                                color = self.anim_params["color2"],
                                line_width = 3
                                )

def update_animation_time(time):
    global animation_time
    animation_time = time['new']
    RRL.redraw()
    RRC.redraw()
    RRLC.redraw()
    PRL.redraw()
    PRC.redraw()
    PRLC.redraw()

animation_time_slider = widgets.IntSlider(max=1500,
                                          description=r"$\tau$",
                                          readout = False)
animation_time_slider.observe(update_animation_time, 'value')

play_konstruktion = widgets.Play(
    value = 0,
    min = 0,
    max = 1500,
    step = 5,
    interval = 25,
    disabled = False,
    _repeat = True
)

widgets.jslink((play_konstruktion, 'value'), (animation_time_slider, 'value'))
control = HBox([animation_time_slider,
                play_konstruktion])

RRL  = KWSR_Zeigerkonstruktion(dtype = "RRL")
RRC  = KWSR_Zeigerkonstruktion(dtype = "RRC")
RRLC = KWSR_Zeigerkonstruktion(dtype = "RRLC")
PRL  = KWSR_Zeigerkonstruktion(dtype = "PRL")
PRC  = KWSR_Zeigerkonstruktion(dtype = "PRC")
PRLC = KWSR_Zeigerkonstruktion(dtype = "PRLC")

In [4]:
class KWSR_ZDiagramm_real(KWSR_Diagramm):
    
    def __init__(self):
        
        self.dparam = {
            "RRL":{
                "corr_x" :   0,
                "corr_y" :   0,
                "angle_L":  90,
                "angle_C": -90,
                "color1" : "I",
                "color2" : "U"
            },
            "RRC":{
                "corr_x" :   0,
                "corr_y" :   0,
                "angle_L":  90,
                "angle_C": -90,
                "color1" : "I",
                "color2" : "U"
            },
            "RRLC":{
                "corr_x" :   6,
                "corr_y" :   1,
                "angle_L":  90,
                "angle_C": -90,
                "color1" : "I",
                "color2" : "U"
            },
            "PRL":{
                "corr_x" :   0,
                "corr_y" :   0,
                "angle_L": -90,
                "angle_C":  90,
                "color1" : "U",
                "color2" : "I"
            },
            "PRC":{
                "corr_x" :   0,
                "corr_y" :   0,
                "angle_L": -90,
                "angle_C":  90,
                "color1" : "U",
                "color2" : "I"
            },
            "PRLC":{
                "corr_x" :   6,
                "corr_y" :  -1,
                "angle_L": -90,
                "angle_C":  90,
                "color1" : "U",
                "color2" : "I"
            },
        }
        for img in self.dparam:
            file = open("img/" + img + ".png", "rb")
            self.dparam[img]["image"] = file.read()
        
        self.dtype = widgets.Dropdown(
            options=[("RL-Reihenschaltung",       "RRL"),
                     ("RC-Reihenschaltung",       "RRC"),
                     ("RLC-Reihenschwingkreis",   "RRLC"),
                     ("RL-Parallelschaltung",     "PRL"),
                     ("RC-Parallelschaltung",     "PRC"),
                     ("RLC-Parallelschwingkreis", "PRLC")],
            value = "RRL",
            layout = Layout(width="391px"),
            disabled = False,
        )
        self.l_max = 175
        self.dtype.observe(self.reconstruct)
        
        self.reset_button = widgets.Button(description='Zurücksetzen',
                                           disabled=False,
                                           tooltip='Bla')
        self.reset_button.on_click(self.reset_values)
        
        self.RRL  = widgets.Image(value=self.dparam["RRL"]["image"],
                                  format='png')
        self.RRC  = widgets.Image(value=self.dparam["RRC"]["image"],
                                  format='png')
        self.RRLC = widgets.Image(value=self.dparam["RRLC"]["image"],
                                  format='png')
        self.PRL  = widgets.Image(value=self.dparam["PRL"]["image"],
                                  format='png')
        self.PRC  = widgets.Image(value=self.dparam["PRC"]["image"],
                                  format='png')
        self.PRLC = widgets.Image(value=self.dparam["PRLC"]["image"],
                                  format='png')
        
        # Maßstab
        self.scale_slider = widgets.FloatLogSlider(value = 3,
                                                   base = 10,
                                                   min = -6,
                                                   max = 12,
                                                   step = 0.01,
                                                   description = 'Maßstab:',
                                                   readout = False,
                                                   readout_format = '.1f',
                                                   layout = Layout(width="375px")
                                                  )
        self.scale_checkbox = widgets.Checkbox(value = True,
                                               description = 'Automatisch',
                                               indent = False)
        self.scale_slider.observe(self.redraw)
        self.scale_checkbox.observe(self.redraw)
        self.scale_group = HBox([self.scale_slider, self.scale_checkbox])
        
        # R
        self.R_slider = widgets.FloatSlider(min=1, max=999, value=200, description="$R$", readout=False, layout=self.scale_slider.layout)
        self.R_slider.observe(self.redraw, 'value')
        self.R_text = widgets.FloatText(layout=Layout(width="80px"))
        widgets.jslink((self.R_slider, 'value'), (self.R_text, 'value'))
        self.R_unit = widgets.Dropdown(
            options=[("mOhm", 0.001), ("Ohm", 1), ("kOhm", 1000)],
            value=1,
            layout=Layout(width="80px"),
            disabled=False,
        )
        self.R_slider.observe(self.redraw)
        self.R_unit.observe(self.redraw)
        self.R_group = HBox([self.R_slider, self.R_text, self.R_unit])
        
        # L
        self.L_slider = widgets.FloatSlider(min=1, max=999, value=250, description="$L$", readout=False, layout=self.scale_slider.layout)
        self.L_slider.observe(self.redraw, 'value')
        self.L_text = widgets.FloatText(layout=self.R_text.layout)
        widgets.jslink((self.L_slider, 'value'), (self.L_text, 'value'))
        self.L_unit = widgets.Dropdown(
            options = [("µH", 0.000001), ("mH", 0.001)],
            value = 0.001,
            layout = self.R_unit.layout,
            disabled = False,
        )
        self.L_slider.observe(self.redraw)
        self.L_unit.observe(self.redraw)
        self.L_group = HBox([self.L_slider, self.L_text, self.L_unit])
        
        # C
        self.C_slider = widgets.FloatSlider(min=1, max=999, value=10, description="$C$", readout=False, layout=self.scale_slider.layout)
        self.C_slider.observe(self.redraw, 'value')
        self.C_text = widgets.FloatText(layout=self.R_text.layout)
        widgets.jslink((self.C_slider, 'value'), (self.C_text, 'value'))
        self.C_unit = widgets.Dropdown(
            options = [("pF", 0.000000000001), ("nF", 0.000000001), ("µF", 0.000001), ("mF", 0.001)],
            value = 0.000001,
            layout = self.R_unit.layout,
            disabled = False,
        )
        self.C_slider.observe(self.redraw)
        self.C_unit.observe(self.redraw)
        self.C_group = HBox([self.C_slider, self.C_text, self.C_unit])
        
        # f
        self.f_slider = widgets.FloatSlider(min=1, max=999, value="50", description="$f$", readout=False, layout=self.scale_slider.layout)
        self.f_text = widgets.FloatText(layout=self.R_text.layout)
        widgets.jslink((self.f_slider, 'value'), (self.f_text, 'value'))
        self.f_unit = widgets.Dropdown(
            options = [("Hz", 1), ("kHz", 1000), ("MHz", 1000000), ("GHz", 1000000000)],
            value = 1,
            layout = self.R_unit.layout,
            disabled = False,
        )
        self.f_slider.observe(self.redraw)
        self.f_unit.observe(self.redraw)
        self.f_group = HBox([self.f_slider, self.f_text, self.f_unit])
        
        # omega
        self.omega_text = widgets.FloatText(
            description = r"$\omega$",
            disabled = True,
            layout = Layout(width = "260px")
        )
        self.omega_label = widgets.Label(value = r"$\,1 / s$")
        self.omega_group = HBox([self.omega_text, self.omega_label])
        
        self.I_slider = widgets.FloatSlider(min=1,
                                            max=100,
                                            value="2",
                                            description="$\\color{%s}{\\rightarrow \\underline{I}}$" % colors["I"],
                                            readout=False,
                                            layout=Layout(width="459px"))
        self.I_slider.observe(self.redraw)        
        self.I_text = widgets.FloatText(layout=self.R_text.layout)
        widgets.jslink((self.I_slider, 'value'), (self.I_text, 'value'))
        self.I_label = widgets.Label(value = r"$\,A$")
        self.I_group = HBox([self.I_slider, self.I_text, self.I_label])
        
        # ZR
        self.ZR_text = widgets.FloatText(
            description = r"$\underline{Z}_{R}$",
            disabled = True,
            layout = self.omega_text.layout
        )
        self.ZR_label = widgets.Label(value = r"$\,\Omega$")
        self.UR_text = widgets.FloatText(
            description = r"$\color{%s}{\rightarrow \underline{U}_{R}}$" % colors["U"],
            disabled = True,
            layout = self.omega_text.layout
        )
        self.UR_label = widgets.Label(value = r"$\,V$")
        self.ZR_group = HBox([self.ZR_text, self.ZR_label, self.UR_text, self.UR_label])
        
        # ZL
        self.ZL_text = widgets.FloatText(
            description = r"$\underline{Z}_L$",
            disabled = True,
            layout = self.omega_text.layout
        )
        self.ZL_label = widgets.Label(value = r"$\,\Omega$")
        self.UL_text = widgets.FloatText(
            description = r"$\color{%s}{\uparrow \underline{U}_{L}}$" % colors["U"],
            disabled = True,
            layout = self.omega_text.layout
        )
        self.UL_label = widgets.Label(value = r"$\,V$")
        self.ZL_group = HBox([self.ZL_text, self.ZL_label, self.UL_text, self.UL_label])
        
        # ZC
        self.ZC_text = widgets.FloatText(
            description = r"$\underline{Z}_C$",
            disabled = True,
            layout = self.omega_text.layout
        )
        self.ZC_label = widgets.Label(value = r"$\,\Omega$")
        self.UC_text = widgets.FloatText(
            description = r"$\color{%s}{\downarrow \underline{U}_{C}}$" % colors["U"],
            disabled = True,
            layout = self.omega_text.layout
        )
        self.UC_label = widgets.Label(value = r"$\,V$")
        self.ZC_group = HBox([self.ZC_text, self.ZC_label, self.UC_text, self.UC_label])
        
        # Z
        self.Z_text = widgets.FloatText(
            description = r"$\underline{Z}$",
            disabled = True,
            layout = self.omega_text.layout
        )
        self.Z_label = widgets.Label(value = r"$\,\Omega$")
        self.Ures_text = widgets.FloatText(
            description = r"$\color{%s}{\underline{U}}$" % colors["U"],
            disabled = True,
            layout = self.omega_text.layout
        )
        self.Ures_label = widgets.Label(value = r"$\,V$")
        self.Z_group = HBox([self.Z_text, self.Z_label, self.Ures_text, self.Ures_label])

        self.U_slider = widgets.FloatSlider(min = 1,
                                          max = 999,
                                          value = "230",
                                          description="$\\color{%s}{\\rightarrow \\underline{U}}$" % colors["U"],
                                          readout = False)
        self.U_slider.observe(self.redraw)        
        self.U_text = widgets.FloatText(layout=self.R_text.layout)
        self.U_label = widgets.Label(value = r"$\,V$")
        widgets.jslink((self.U_slider, 'value'), (self.U_text, 'value'))
        self.U_group = HBox([self.U_slider, self.U_text, self.U_label])
        
        # YR
        self.YR_text = widgets.FloatText(
            description = r"$\underline{Y}_R$",
            disabled = True,
            layout = self.omega_text.layout
        )
        self.YR_label = widgets.Label(value = r"$\,S$")
        self.IR_text = widgets.FloatText(
            description = r"$\color{%s}{\rightarrow \underline{I}_{R}}$" % colors["I"],
            disabled = True,
            layout = self.omega_text.layout
        )
        self.IR_label = widgets.Label(value = r"$\,A$")        
        self.YR_group = HBox([self.YR_text, self.YR_label, self.IR_text, self.IR_label])
        
        # YL
        self.YL_text = widgets.FloatText(
            description = r"$\underline{Y}_L$",
            disabled = True,
            layout = self.omega_text.layout
        )
        self.YL_label = widgets.Label(value = r"$\,S$")
        self.IL_text = widgets.FloatText(
            description = r"$\color{%s}{\downarrow \underline{I}_{L}}$" % colors["I"],
            disabled = True,
            layout = self.omega_text.layout
        )
        self.IL_label = widgets.Label(value = r"$\,A$")        
        self.YL_group = HBox([self.YL_text, self.YL_label, self.IL_text, self.IL_label])
        
        # YC
        self.YC_text = widgets.FloatText(
            description = r"$\underline{Y}_C$",
            disabled = True,
            layout = self.omega_text.layout
        )
        self.YC_label = widgets.Label(value = r"$\,S$")
        self.IC_text = widgets.FloatText(
            description = r"$\color{%s}{\uparrow \underline{I}_{C}}$" % colors["I"],
            disabled = True,
            layout = self.omega_text.layout
        )
        self.IC_label = widgets.Label(value = r"$\,A$")        
        self.YC_group = HBox([self.YC_text, self.YC_label, self.IC_text, self.IC_label])
        
        # Y
        self.Y_text = widgets.FloatText(
            description = r"$\underline{Y}$",
            disabled = True,
            layout = self.omega_text.layout
        )
        self.Y_label = widgets.Label(value = r"$\,S$")
        self.Ires_text = widgets.FloatText(
            description = r"$\color{%s}{\underline{I}}$" % colors["I"],
            disabled = True,
            layout = self.omega_text.layout
        )
        self.Ires_label = widgets.Label(value = r"$\,A$")        
        self.Y_group = HBox([self.Y_text, self.Y_label, self.Ires_text, self.Ires_label])
        
        # phi
        self.phi_text = widgets.FloatText(
            description = r"$\varphi$",
            disabled = True,
            layout = self.omega_text.layout
        )
        self.phi_label = widgets.Label(value = r"$\,^{\circ}$")
        self.phi_group = HBox([self.phi_text, self.phi_label])
        
        super().__init__(250, 450, 5, None, None)
        display(HBox([
                VBox([self.RRL,
                      self.RRC,
                      self.RRLC,
                      self.PRL,
                      self.PRC,
                      self.PRLC,
                      self.canvas
                     ]),
                VBox([HBox([self.dtype, self.reset_button]),
                      self.scale_group,
                      self.R_group,
                      self.L_group,
                      self.C_group,
                      self.f_group,
                      self.omega_group,
                      self.I_group,
                      self.ZR_group,
                      self.ZL_group,
                      self.ZC_group,
                      self.Z_group,
                      self.U_group,
                      self.YR_group,
                      self.YL_group,
                      self.YC_group,
                      self.Y_group,
                      self.phi_group
                      ])
                ]))
        self.reconstruct()
        self.redraw()
    
    def reset_values(self, *args, **kwarg):
        self.scale_checkbox.value = True
        self.R_slider.value = 200
        self.R_unit.value   = 1
        self.L_slider.value = 250
        self.L_unit.value   = 0.001
        self.C_slider.value = 10
        self.C_unit.value   = 0.000001
        self.f_slider.value = 50
        self.f_unit.value   = 1
        self.U_slider.value = 230
        self.I_slider.value = 2
        self.redraw()
        
    def draw_background(self):
        bg = self.canvas[0]
        with hold_canvas(bg):
            
            # Abszisse
            self.draw_arrow(layer = bg,
                            xy = (25, 225),
                            angle = 0,
                            length = 200,
                            color = colors["black"],
                            line_width = 1
                           )
            bg.draw_image(img_Re, 215, 200)
            
            # Ordinate
            self.draw_arrow(layer = bg,
                            xy = (25, 425),
                            angle = 90,
                            length = 400,
                            color = colors["black"],
                            line_width = 1
                           )
            bg.draw_image(img_j_Im, 35, 10)
    
    def reconstruct(self, *args, **kwargs):
        if self.dtype.value[0] == "R":
            self.canvas[0].draw_image(img_I, 215, 235)
        else:
            self.canvas[0].draw_image(img_U, 215, 235)
        if self.dtype.value  == "RRL":
            self.L_group.layout.visibility     = "visible"
            self.C_group.layout.visibility     = "hidden"
            self.I_group.layout.visibility     = "visible"
            self.ZR_group.layout.visibility    = "visible"
            self.ZL_group.layout.visibility    = "visible"
            self.ZC_group.layout.visibility    = "hidden"
            self.Z_group.layout.visibility     = "visible"
            self.U_group.layout.visibility     = "hidden"
            self.YR_group.layout.visibility    = "hidden"
            self.YL_group.layout.visibility    = "hidden"
            self.YC_group.layout.visibility    = "hidden"
            self.Y_group.layout.visibility     = "hidden"
            self.RRL.layout.display            = "block"
            self.RRC.layout.display            = "none"
            self.RRLC.layout.display           = "none"
            self.PRL.layout.display            = "none"
            self.PRC.layout.display            = "none"
            self.PRLC.layout.display           = "none"
        elif self.dtype.value == "RRC":
            self.L_group.layout.visibility     = "hidden"
            self.C_group.layout.visibility     = "visible"
            self.I_group.layout.visibility     = "visible"
            self.ZR_group.layout.visibility    = "visible"
            self.ZL_group.layout.visibility    = "hidden"
            self.ZC_group.layout.visibility    = "visible"
            self.Z_group.layout.visibility     = "visible"
            self.U_group.layout.visibility     = "hidden"
            self.YR_group.layout.visibility    = "hidden"
            self.YL_group.layout.visibility    = "hidden"
            self.YC_group.layout.visibility    = "hidden"
            self.Y_group.layout.visibility     = "hidden"
            self.RRL.layout.display            = "none"
            self.RRC.layout.display            = "block"
            self.RRLC.layout.display           = "none"
            self.PRL.layout.display            = "none"
            self.PRC.layout.display            = "none"
            self.PRLC.layout.display           = "none"
        elif self.dtype.value == "RRLC":
            self.L_group.layout.visibility     = "visible"
            self.C_group.layout.visibility     = "visible"
            self.I_group.layout.visibility     = "visible"
            self.ZR_group.layout.visibility    = "visible"
            self.ZL_group.layout.visibility    = "visible"
            self.ZC_group.layout.visibility    = "visible"
            self.Z_group.layout.visibility     = "visible"
            self.U_group.layout.visibility     = "hidden"
            self.YR_group.layout.visibility    = "hidden"
            self.YL_group.layout.visibility    = "hidden"
            self.YC_group.layout.visibility    = "hidden"
            self.Y_group.layout.visibility     = "hidden"
            self.RRL.layout.display            = "none"
            self.RRC.layout.display            = "none"
            self.RRLC.layout.display           = "block"
            self.PRL.layout.display            = "none"
            self.PRC.layout.display            = "none"
            self.PRLC.layout.display           = "none"
        elif self.dtype.value == "PRL":
            self.L_group.layout.visibility     = "visible"
            self.C_group.layout.visibility     = "hidden"
            self.I_group.layout.visibility     = "hidden"
            self.ZR_group.layout.visibility    = "hidden"
            self.ZL_group.layout.visibility    = "hidden"
            self.ZC_group.layout.visibility    = "hidden"
            self.Z_group.layout.visibility     = "hidden"
            self.U_group.layout.visibility     = "visible"
            self.YR_group.layout.visibility    = "visible"
            self.YL_group.layout.visibility    = "visible"
            self.YC_group.layout.visibility    = "hidden"
            self.Y_group.layout.visibility     = "visible"
            self.RRL.layout.display            = "none"
            self.RRC.layout.display            = "none"
            self.RRLC.layout.display           = "none"
            self.PRL.layout.display            = "block"
            self.PRC.layout.display            = "none"
            self.PRLC.layout.display           = "none"
        elif self.dtype.value == "PRC":
            self.L_group.layout.visibility     = "hidden"
            self.C_group.layout.visibility     = "visible"
            self.I_group.layout.visibility     = "hidden"
            self.ZR_group.layout.visibility    = "hidden"
            self.ZL_group.layout.visibility    = "hidden"
            self.ZC_group.layout.visibility    = "hidden"
            self.Z_group.layout.visibility     = "hidden"
            self.U_group.layout.visibility     = "visible"
            self.YR_group.layout.visibility    = "visible"
            self.YL_group.layout.visibility    = "hidden"
            self.YC_group.layout.visibility    = "visible"
            self.Y_group.layout.visibility     = "visible"
            self.RRL.layout.display            = "none"
            self.RRC.layout.display            = "none"
            self.RRLC.layout.display           = "none"
            self.PRL.layout.display            = "none"
            self.PRC.layout.display            = "block"
            self.PRLC.layout.display           = "none"
        elif self.dtype.value == "PRLC":
            self.L_group.layout.visibility     = "visible"
            self.C_group.layout.visibility     = "visible"
            self.I_group.layout.visibility     = "hidden"
            self.ZR_group.layout.visibility    = "hidden"
            self.ZL_group.layout.visibility    = "hidden"
            self.ZC_group.layout.visibility    = "hidden"
            self.Z_group.layout.visibility     = "hidden"
            self.U_group.layout.visibility     = "visible"
            self.YR_group.layout.visibility    = "visible"
            self.YL_group.layout.visibility    = "visible"
            self.YC_group.layout.visibility    = "visible"
            self.Y_group.layout.visibility     = "visible"
            self.RRL.layout.display            = "none"
            self.RRC.layout.display            = "none"
            self.RRLC.layout.display           = "none"
            self.PRL.layout.display            = "none"
            self.PRC.layout.display            = "none"
            self.PRLC.layout.display           = "block"
        self.redraw()


    def redraw(self, *args, **kwargs):
        
        # Darstellung der eingestellten Werte
        omega = 2 * pi * self.f_slider.value * self.f_unit.value
        self.omega_text.value = omega
        R = self.R_slider.value * self.R_unit.value
        L = self.L_slider.value * self.L_unit.value
        C = self.C_slider.value * self.C_unit.value
        
        # Berechnung der neuen Werte
        if self.dtype.value[0] == "R":
            self.ZR_text.value = R
            UR = R * self.I_slider.value
            self.UR_text.value = UR
            ZL = omega * self.L_slider.value * self.L_unit.value
            self.ZL_text.value = ZL
            UL = ZL * self.I_slider.value
            self.UL_text.value = UL
            ZC = 1 / (omega * self.C_slider.value * self.C_unit.value)
            self.ZC_text.value = ZC
            UC = ZC * self.I_slider.value
            self.UC_text.value = UC
            if self.dtype.value == "RRL":
                YZ = sqrt(R**2 + ZL**2)
                phi = atan(omega * L / R)
            elif self.dtype.value == "RRC":
                YZ = sqrt(R**2 + ZC**2)
                phi = atan((-1 / (omega * C)) / R)
            elif self.dtype.value == "RRLC":
                YZ = sqrt(R**2 + (ZL - ZC)**2)
                phi = atan((omega * L - 1 / (omega * C)) / R)
            self.Z_text.value = YZ
            fac = self.I_slider.value
            res = YZ * fac
            self.Ures_text.value = res
        elif self.dtype.value[0] == "P":
            YR = 1 / R
            self.YR_text.value = YR
            IR = YR * self.U_slider.value
            self.IR_text.value = IR
            YL = 1 / (omega * self.L_slider.value * self.L_unit.value)
            self.YL_text.value = YL
            IL = YL * self.U_slider.value
            self.IL_text.value = IL
            YC = omega * self.C_slider.value * self.C_unit.value
            self.YC_text.value = YC
            IC = YC * self.U_slider.value
            self.IC_text.value = IC
            if self.dtype.value == "PRL":
                YZ = sqrt(YR**2 + YL**2)
                phi = atan((-1 / (omega * L)) / (1 / R))
            elif self.dtype.value == "PRC":
                YZ = sqrt(YR**2 + YC**2)
                phi = atan((omega * C) / (1 / R))
            elif self.dtype.value == "PRLC":
                YZ = sqrt(YR**2 + (-YL + YC)**2)
                phi = atan(((-1 / (omega * L) + omega * C) / (1 / R)))
            self.Y_text.value = YZ
            fac = self.U_slider.value
            res = YZ * fac
            self.Ires_text.value = res
        
        # Maßstab berücksichtigen
        if self.scale_checkbox.value == True:
            corr = YZ * fac / self.l_max
            self.scale_slider.value = YZ * fac
        else:
            corr = self.scale_slider.value / self.l_max
        self.phi_text.value = phi * 180 / pi

        if self.dtype.value[0] == "R":
            R_len   = R  / corr * fac
            L_len   = ZL / corr * fac
            C_len   = ZC / corr * fac
            res_len = YZ / corr * fac
            angle   = phi * 180 / pi
        else:
            R_len   = YR / corr * fac
            L_len   = YL / corr * fac
            C_len   = YC / corr * fac
            res_len = YZ / corr * fac
            angle   = phi * 180 / pi

        xy = (res_len * cos(phi), res_len * sin(phi))
        

        # Darstellung Zeigerdiagramm
        with hold_canvas(self.canvas[1]):
            self.canvas[1].clear()
            self.draw_arrow(layer = self.canvas[1],
                            xy = (25, 225),
                            angle = 0,
                            length = R_len,
                            color = colors[self.dparam[self.dtype.value]["color2"]],
                            line_width = 1,
                            tip_width = 8)
        # L
        if self.dtype.value in ["RRL", "RRLC", "PRL", "PRLC"]:
            if self.dtype.value in ["RRL", "PRL"]:
                with hold_canvas(self.canvas[3]):
                    self.canvas[3].clear()
            with hold_canvas(self.canvas[2]):
                self.canvas[2].clear()
                self.draw_arrow(layer = self.canvas[2],
                                xy = (25 + xy[0] + self.dparam[self.dtype.value]["corr_x"], 225),
                                angle = self.dparam[self.dtype.value]["angle_L"],
                                length = L_len,
                                color = colors[self.dparam[self.dtype.value]["color2"]],
                                line_width = 1,
                                tip_width = 8)
        # C
        if self.dtype.value in ["RRC", "RRLC", "PRC", "PRLC"]:
            if self.dtype.value in ["RRC", "PRC"]:
                with hold_canvas(self.canvas[2]):
                    self.canvas[2].clear()
            with hold_canvas(self.canvas[3]):
                self.canvas[3].clear()
                self.draw_arrow(layer = self.canvas[3],
                                xy = (25 + xy[0], 225 - self.dparam[self.dtype.value]["corr_y"] * L_len),
                                angle = self.dparam[self.dtype.value]["angle_C"],
                                length = C_len,
                                color = colors[self.dparam[self.dtype.value]["color2"]],
                                line_width = 1,
                                tip_width = 8)
        # Gesamt
        with hold_canvas(self.canvas[4]):
            self.canvas[4].clear()
            self.draw_arrow(layer = self.canvas[4],
                            xy = (25, 225),
                            angle = angle,
                            length = res_len,
                            color = colors[self.dparam[self.dtype.value]["color2"]],
                            line_width = 3)

In [5]:
class KWSR_KombiDiagramm(KWSR_Diagramm):
    
    def __init__(self, link):
        
        self.link = link
        self.link.phi_text.observe(self.redraw, 'value')
        self.alpha = 0
        self.phi = self.link.phi_text.value
                
        # Bedienelemente
        self.alpha_slider = widgets.IntSlider(max=360,
                                              description=r"$\alpha$")
        self.alpha_slider.observe(self.update_alpha, 'value')
        self.phi_slider = widgets.IntSlider(min=-90,
                                       max=90,
                                       description=r"$\varphi$")
        self.phi_slider.observe(self.update_phi, 'value')
        self.phi_checkbox = widgets.Checkbox(value = True,
                                             description = r'$\varphi$ aus realem Zeigerdiagramm')
        self.phi_checkbox.observe(self.redraw, 'value')
        self.play = widgets.Play(
            value = 0,
            min = 0,
            max = 360,
            step = 3,
            interval = 20,
            description = "Press play",
            disabled = False,
            _repeat = True
        )
        widgets.jslink((self.play, 'value'), (self.alpha_slider, 'value'))
        super().__init__(800, 300, 5)
        display(self.canvas)
        display(self.alpha_slider)
        display(self.phi_slider)
        display(self.phi_checkbox)
        widgets.jslink((self.phi_checkbox, 'value'), (self.phi_slider, 'disabled'))
        display(self.play)

    def update_alpha(self, angle):
        self.alpha = angle['new']
        self.redraw()

    def update_phi(self, angle):
        self.phi = angle['new']
        self.redraw()
                
    def draw_background(self):
        bg = self.canvas[0]
        with hold_canvas(bg):
            
            # Abszisse
            self.draw_arrow(layer = bg,
                            xy = (25, 150),
                            angle = 0,
                            length = 425,
                            color = colors["black"],
                            line_width = 1
                           )
            bg.draw_image(img_omega_t, 440, 125)
            
            # Ordinate
            self.draw_arrow(layer = bg,
                            xy = (25, 275),
                            angle = 90,
                            length = 250,
                            color = colors["black"],
                            line_width = 1
                           )
            bg.draw_image(img_u_t, 40, 10)
            bg.draw_image(img_i_t, 40, 30)
    
            # Zeigerdiagramm für Kreise
            bg.fill_style = colors["black"]
            self.draw_arrow(layer = bg,
                            xy = (525, 150),
                            angle = 0,
                            length = 250,
                            color = colors["black"],
                            line_width = 1
                           )
            bg.draw_image(img_Re, 765, 125)
            
            # y
            self.draw_arrow(layer = bg,
                            xy = (650, 275),
                            angle = 90,
                            length = 250,
                            color = colors["black"],
                            line_width = 1
                           )
            bg.draw_image(img_j_Im, 665, 10)
                
            # Spannungskurve
            bg.line_width = 2
            bg.begin_path()
            bg.stroke_style = colors["U"]
            bg.move_to(25,150)
            for xi in range(401):
                bg.line_to(xi + 25,
                           -100 * sin(2 * pi * xi / 400) + 150)
            bg.stroke()
            
            bg.line_width = 1
            
            # Spannungskreis
            bg.stroke_arc(650, 150, 100, 0, 2 * pi)
            
            # Stromkreis
            bg.stroke_style = colors["I"]
            bg.stroke_arc(650, 150, 61.8, 0, 2 * pi)
    
    def redraw(self, *args, **kwargs):
       
        with hold_canvas(self.canvas[2]):
            self.canvas[2].stroke_style = colors["U"]
            
            alpha = self.alpha
            if self.phi_checkbox.value == True:
                phi = self.link.phi_text.value
                self.phi_slider.value = self.link.phi_text.value
            else:
                phi = self.phi
            
            xc = 100 * sin((self.alpha + 90) * pi / 180)
            yc = 100 * cos((self.alpha + 90) * pi / 180)
            
            # Spannungszeiger
            self.canvas[2].clear()
            self.draw_arrow(layer = self.canvas[2],
                            xy = (650, 150),
                            angle = alpha,
                            length = 100,
                            color = colors["U"],
                            line_width = 3
                           )

        with hold_canvas(self.canvas[4]):
            xc2 = 61.2 * sin((alpha + 90 + phi) * pi / 180)
            yc2 = 61.2 * cos((alpha + 90 + phi) * pi / 180)
            
            # Cursor
            self.canvas[4].line_width = 3
            self.canvas[4].begin_path()

            self.canvas[4].clear()
            self.canvas[4].fill_style = colors["U"]
            self.canvas[4].fill_arc(25 + 400 * alpha / 360, 150 + yc, 5, 0, 2 * pi)
            self.canvas[4].fill_style = colors["I"]
            self.canvas[4].fill_arc(25 + 400 * alpha / 360, 150 + yc2, 5, 0, 2 * pi)
            
        with hold_canvas(self.canvas[1]):
            self.canvas[1].clear()
            # Verbindungspunktlinien
            self.canvas[1].line_width = 1
            self.canvas[1].set_line_dash([1, 3])
            self.canvas[1].stroke_style = colors["U"]
            self.canvas[1].begin_path()
            self.canvas[1].move_to(25 + 400 * alpha / 360, 150 + yc)
            self.canvas[1].line_to(650 + xc, 150 + yc)
            self.canvas[1].stroke()

            self.canvas[1].stroke_style = colors["I"]
            self.canvas[1].begin_path()
            self.canvas[1].move_to(25 + 400 * alpha / 360, 150 + yc2)
            self.canvas[1].line_to(650 + xc2, 150 + yc2)
            self.canvas[1].stroke()

        
        with hold_canvas(self.canvas[3]):
            
            # Stromkurve
            angle = phi * pi / 180
            self.canvas[3].clear()
            self.canvas[3].begin_path()
            self.canvas[3].line_width = 2
            self.canvas[3].stroke_style = colors["I"]
            self.canvas[3].move_to(25,150 - 61.8 * sin(angle))
            for xi in range(401):
                self.canvas[3].line_to(xi + 25,
                               -61.8 * sin(2 * pi * xi / 400 + angle) + 150)
            self.canvas[3].stroke()
            
            # Stromzeiger
            self.draw_arrow(layer = self.canvas[3],
                            xy = (650, 150),
                            angle = phi + alpha,
                            length = 61.8,
                            color = colors["I"],
                            line_width = 3
                           )

## Theorie

### Reihenschaltung

<table width="80%">
    <tr>
        <td align="center">
            <b>R-L-Reihenschaltung</b>
        </td>
        <td align="center">
            <b>R-C-Reihenschaltung</b>
        </td>
        <td align="center">
            <b>R-L-C-Reihenschwingkreis</b>
        </td>
    </tr>
    <tr>
        <td width=33% valign="top" align="center">
            <img src="img/RRL.png">
        </td>
        <td width=33% valign="top" align="center">
            <img src="img/RRC.png">
        </td>
        <td width=33% valign="top" align="center">
            <img src="img/RRLC.png">
        </td>
    </tr>
    <tr>
        <td width="33%" valign="top">
            $$
            \begin{split}
            \underline{Z}_{R-L} & = R + \underline{Z}_L \\
            & = \underbrace{R}_{Re} + j \cdot \underbrace{\omega \cdot L}_{Im} \\
            \varphi_{Z,R-L} & = \arctan{\left( \frac{\omega \cdot L}{R}\right)}\\
            \end{split}
            $$
        </td>
        <td width="33%" valign="top">
            $$
            \begin{split}
            \underline{Z}_{R-C} & = R + \underline{Z}_C \\
            & = R + \frac{1}{j \cdot \omega \cdot C} \\
            & = \underbrace{R}_{Re} + j \cdot \underbrace{-\,\frac{1}{\omega \cdot C}}_{Im} \\
            \varphi_{Z,R-C} & = \arctan{\left( \frac{-1 / (\omega \cdot C)}{R}\right)}
            \end{split}
            $$
        </td>
        <td width=33% valign="top">
            $$
            \begin{split}
            \underline{Z}_{R-L-C} & = R + \underline{Z}_C + \underline{Z}_L \\
            & = R + j \cdot \omega \cdot L + \frac{1}{j \cdot \omega \cdot C} \\
            & = \underbrace{R}_{Re} + j \cdot \underbrace{\left(\omega \cdot L -\,\frac{1}{\omega \cdot C} \right)}_{Im} \\
            \varphi_{Z,R-L-C} & = \arctan{\left( \frac{\omega \cdot L -1 / (\omega \cdot C)}{R}\right)}
            \end{split}
            $$
        </td>
    </tr>
</table>

In [6]:
display(
    VBox([
        HBox([Box([], layout=Layout(width="30%")),
              Box([RRL.canvas],  layout=Layout(width="20%")),
              Box([], layout=Layout(width="30%")),
              Box([RRC.canvas],  layout=Layout(width="20%")),
              Box([], layout=Layout(width="30%")),
              Box([RRLC.canvas], layout=Layout(width="20%"))
             ],layout=Layout(width="80%")),
        HBox([Box([], layout=Layout(width="30%")),
              control])
    ])
)

### Parallelschaltung

<table width="80%">
    <tr>
        <td align="center">
            <b>R-L-Parallelschaltung</b>
        </td>
        <td align="center">
            <b>R-C-Parallelschaltung</b>
        </td>
        <td align="center">
            <b>R-L-C-Parallelschwingkreis</b>
        </td>
    </tr>
    <tr>
        <td width=33% valign="top" align="center">
            <img src="img/PRL.png">
        </td>
        <td width=33% valign="top" align="center">
            <img src="img/PRC.png">
        </td>
        <td width=33% valign="top" align="center">
            <img src="img/PRLC.png">
        </td>
    </tr>
    <tr>
        <td width="33%" valign="top">
            $$
            \begin{split}
            \underline{Y}_{R \parallel L} & = \frac{1}{R} + \underline{Y}_L \\
            & = \frac{1}{R} + \frac{1}{j \cdot \omega \cdot L} \\
            & = \underbrace{\frac{1}{R}}_{Re} + j \cdot \underbrace{-\,\frac{1}{\omega \cdot L}}_{Im} \\            
            \varphi_{Y,R \parallel L} & = \arctan{\left( \frac{-1 / (\omega \cdot L)}{1/R}\right)}
            \end{split}
            $$
        </td>
        <td width="33%" valign="top">
            $$
            \begin{split}
            \underline{Y}_{R \parallel C} & = \frac{1}{R} + \underline{Y}_C \\
            & = \underbrace{\frac{1}{R}}_{Re} + {j \cdot \underbrace{\omega \cdot C}_{Im}} \\
            \varphi_{Y,R \parallel C} & = \arctan{\left( \frac{\omega \cdot C}{1/R}\right)}
            \end{split}
            $$
        </td>
        <td width=33% valign="top">
            $$
            \begin{split}
            \underline{Y}_{R \parallel L \parallel C} & = \frac{1}{R} + \underline{Y}_L + \underline{Y}_C \\
            & = \frac{1}{R} + j \cdot \frac{1}{j \cdot \omega \cdot L} + j \cdot \omega \cdot C \\
            & = \underbrace{\frac{1}{R}}_{Re} + j \cdot \underbrace{\left(- \frac{1}{\omega \cdot L} + \omega \cdot C \right)}_{Im} \\
            \varphi_{Y,R \parallel L \parallel C} & = \arctan{\left( \frac{-1 / (\omega \cdot L) + \omega \cdot C}{1/R}\right)}
            \end{split}
            $$
        </td>
    </tr>
</table>

In [7]:
display(
    VBox([
        HBox([Box([], layout=Layout(width="30%")),
              Box([PRL.canvas],  layout=Layout(width="20%")),
              Box([], layout=Layout(width="30%")),
              Box([PRC.canvas],  layout=Layout(width="20%")),
              Box([], layout=Layout(width="30%")),
              Box([PRLC.canvas], layout=Layout(width="20%"))
             ],layout=Layout(width="80%")),
        HBox([Box([], layout=Layout(width="30%")),
              control])
    ])
)

## Praxis

In [8]:
KWSR_ZDR = KWSR_ZDiagramm_real()
KWSR_KD = KWSR_KombiDiagramm(link=KWSR_ZDR)

MultiCanvas(height=300, width=800)

IntSlider(value=0, description='$\\alpha$', max=360)

IntSlider(value=21, description='$\\varphi$', max=90, min=-90)

Checkbox(value=True, description='$\\varphi$ aus realem Zeigerdiagramm')

Play(value=0, description='Press play', interval=20, max=360, step=3)